In [1]:
import numpy as np
import meshplot as mp
import meshio
import triangle

# Square

In [2]:
def generate_isotropic_square_boundary(n=50):
    points = []
    for x in np.linspace(0, 1, n):
        for y in [0, 1]:
            points.append([x, y])
    for y in np.linspace(0, 1, n):
        for x in [0, 1]:
            points.append([x, y])
    points = np.array(points) - [0.5, 0.5] # center the square
    points = np.unique(points, axis=0)
    return points

def generate_isotropic_square(n=50, allow_steiner=True):
    points = generate_isotropic_square_boundary(n)
    max_area = 1 / n**2
    tri = triangle.triangulate({"vertices": points}, f'{"" if allow_steiner else "Y"}a{max_area:f}q')
    return meshio.Mesh(tri["vertices"], [("triangle", tri["triangles"])])

In [3]:
def generate_anisotropic_square(nx=50, p=2, min_ny=4, max_ny=None, allow_steiner=True):
    points = []
    if max_ny is None:
        max_ny = nx
    for x in np.linspace(0, 1, nx):
        ny = int((max_ny - min_ny) * (1 - x) + min_ny)
        for y in np.linspace(0, 1, ny):
            points.append([x**p, y])
    points = np.array(points) - [0.5, 0.5] # center the square

    tri = triangle.triangulate({"vertices": points}, f'{"" if allow_steiner else "Y"}q')
    return meshio.Mesh(tri["vertices"], [("triangle", tri["triangles"])])

In [4]:
def plot_mesh(mesh, **kwargs):
    shading = {
        "flat": True, # Flat or smooth shading of triangles
        "wireframe": True, "wire_width": 0.1, "wire_color": "black", # Wireframe rendering
        "width": 600, "height": 600, # Size of the viewer canvas
        "antialias": True, # Antialising, might not work on all GPUs
        "scale": 2.0, # Scaling of the model
        "side": "DoubleSide", # FrontSide, BackSide or DoubleSide rendering of the triangles
        "colormap": "viridis", "normalize": [None, None], # Colormap and normalization for colors
        "background": "#ffffff", # Background color of the canvas
        "line_width": 2.0, "line_color": "black", # Line properties of overlay lines
        "bbox": False, # Enable plotting of bounding box
        "point_color": "red", "point_size": 0.05 # Point properties of overlay points
    }
    shading.update(kwargs)
    return mp.plot(mesh.points, mesh.cells[0].data, shading=shading)

In [5]:
for n in [2, 11, 21, 31, 41, 51]:
    n = int(n)
    isotropic_square = generate_isotropic_square(n=n)
    isotropic_square.write(f"../meshes/square/isotropic_h={1/(n-1):g}.msh", file_format="gmsh")

In [6]:
# plot_mesh(isotropic_square)

In [7]:
anisotropic_square = generate_anisotropic_square(max_ny=100)
anisotropic_square.write("../meshes/square/anisotropic.msh", file_format="gmsh")
# plot_mesh(anisotropic_square)

# Circle

In [8]:
def generate_isotropic_circle(h, allow_steiner=True):
    r = 0.5
    n = int(np.ceil(np.pi / np.arctan2(h, 2 * r)))
    theta = np.linspace(0, 2 * np.pi, n, endpoint=False).reshape(-1, 1)
    points = np.hstack([r * np.cos(theta), r * np.sin(theta)])
    max_area = 1/2 * h**2
    tri = triangle.triangulate({"vertices": points}, f'{"" if allow_steiner else "Y"}a{max_area:f}q')
    return meshio.Mesh(tri["vertices"], [("triangle", tri["triangles"])])

In [9]:
def generate_squircle(n_points=100, radius=0.5, squareness=10, allow_steiner=True):
    # t = np.linspace(0, 2 * np.pi, n_points, endpoint=False).reshape(-1, 1)
    # squareness = 0.99
    # right_term = np.sqrt(1 - np.sqrt(1 - squareness**2 * np.sin(2 * t)**2))
    # x = (radius * np.sign(np.cos(t))) / (squareness * np.sqrt(2) * abs(np.sin(t))) * right_term
    # y = (radius * np.sign(np.sin(t))) / (squareness * np.sqrt(2) * abs(np.cos(t))) * right_term
    # x = radius * np.cos(t)
    # y = radius * np.sin(t)
    # points = np.hstack([x, y])
    points = generate_isotropic_square_boundary(n_points // 4)
    s = np.linalg.norm(points, axis=1, ord=squareness)
    points /= s[:, None]
    max_area = 0.5 * np.linalg.norm(points[1:] - points[0], axis=1).min()**2
    # edges = np.hstack([np.arange(0, points.shape[0] - 1).reshape(-1, 1), np.arange(1, points.shape[0]).reshape(-1, 1)])
    # return points, edges
    tri = triangle.triangulate({"vertices": points}, f'{"" if allow_steiner else "Y"}a{max_area:f}q')
    return meshio.Mesh(tri["vertices"], [("triangle", tri["triangles"])])

In [10]:
for h in (1 / np.arange(10, 60, 10)):
    circle = generate_isotropic_circle(h)
    circle.write(f"../meshes/circle/isotropic_h={h:g}.msh", file_format="gmsh")

In [11]:
# squircle_points, squircle_edges = generate_squircle_boundary()
squircle = generate_squircle(n_points=100, squareness=4)
squircle.write(f"../meshes/squircle.msh", file_format="gmsh")
plot_mesh(squircle)

/home/zachary/.local/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Beam

In [12]:
def generate_beam_boundary(h=0.1):
    width = 4
    height = 0.5
    points = []
    for x in np.linspace(-width/2, width/2, int(np.ceil(width / h))):
        for y in [-height/2, height/2]:
            points.append([x, y])
    for y in np.linspace(-height/2, height/2, int(np.ceil(height / h))):
        for x in [-width/2, width/2]:
            points.append([x, y])
    points = np.unique(points, axis=0)
    return points

def generate_beam(h=0.1, allow_steiner=True):
    points = generate_beam_boundary(h)
    max_area = h**2
    tri = triangle.triangulate({"vertices": points}, f'{"" if allow_steiner else "Y"}a{max_area:f}q')
    return meshio.Mesh(tri["vertices"], [("triangle", tri["triangles"])])

In [20]:
for h in [0.05]:
    beam = generate_beam(h=h)
    beam.write(f"../meshes/beam/isotropic_h={h:g}.msh", file_format="gmsh")
    plot_mesh(beam)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…